<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/11j_Multivariate_Multi_step_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiple Input Multi-Step Output LSTMs**

There are multivariate time series forecasting problems where the output series is separate but dependent upon the input time series, and multiple time steps are required for the output series. 

For example:<br>
[[10 15 25]<br>
[20 25 45]<br>
[30 35 65]<br>
[40 45 85]<br>
[50 55 105]<br>
[60 65 125]<br>
[70 75 145]<br>
[80 85 165]<br>
[90 95 185]<br>

**Input**:<br>
10,15<br>
20,25<br>
30,35<br>
<br>
**Output**:<br>
65<br>
85

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LeakyReLU

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out-1
    # check if we are beyond the dataset
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
in_seq1 = array([10, 21, 32, 43, 54, 65, 76, 87, 98])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]-in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
n_features = X.shape[2]
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)

In [ ]:
# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in,
n_features)))
model.add(LSTM(100, activation='relu')) 
model.add(Dense(n_steps_out)) 
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

**Make Prediction**<br>
Expecting:<br>
3,4

In [ ]:
x_input = array([[76, 75], [87, 85], [98, 95]]) 
x_input = x_input.reshape((1, n_steps_in, n_features)) 
yhat = model.predict(x_input, verbose=0)
print(yhat)

**Assignment:**<br>
1. Change the input sequence to a more complex pattern, does the model still accurately predict?
2. Go out several time steps, what happens to the accuracy?